In [ ]:
import os.path
from os import path

environment = 'Test'

if (path.exists("../Library/cus_databases.ipynb")):
    environment = 'Test'
    Date_Override = 'Y'
else:
    environment = 'Prod'
    Date_Override = 'N'

print('environment is set to: ',environment)

clientCode = 'ALL'
TrainingTypeID = 1
Ranking_ID = 1

def splitter(td):
    td = str(td).split(' ')[-1:][0]

    return td

Max_Round_Robins = 3
print('Round Robin is set to: ',str(Max_Round_Robins))

In [ ]:
import pandas as pd
import sys
import xlsxwriter
import os
import pyodbc 
import numpy as np
from datetime import date, timedelta, datetime
import math

import jinja2
from jinja2 import Environment, FileSystemLoader
import urllib.parse
from pandas import Series, DataFrame
import gc

from IPython.core.display import display, HTML
import json
import urllib
import boto3
from boto.s3.key import Key
import base64

%run ../Library/cus_databases.ipynb
%run ../Library/datetime_utils.ipynb

focus_db = get_db_SS_focus()
focus_dev = get_db_SS_focus_dev()
report_RO_DB = Reporting_Read_Only()

In [ ]:
#Front End Controller

%run ../Library/Environment_Controller.ipynb

time_format = '%Y-%m-%d' 

run_date = datetime.now()
#Date Override
if(Date_Override == 'Y'):
    print('Date Override is On')
    start_date = (run_date.date() - timedelta(days=(run_date).weekday()))+ timedelta(7)

else:
    start_date = (run_date.date() - timedelta(days=(run_date).weekday()))
    Ranking_ID, clientCode, TrainingTypeID, start_date_alt = Environment_control_Scheduling_DateRange(environment,clientCode,TrainingTypeID,Ranking_ID)
    
    if(start_date_alt == 'Next_Week'):
        print('Date is nil, Setting to next Monday from now')
        start_date = start_date + timedelta(7)
        
    elif(start_date_alt == 'Next_2_Week'):
        print('Date is 2 Weeks out, Setting to 2 Monday from now')
        start_date = start_date + timedelta(14)    
        
    elif(start_date_alt == 'Next_3_Week'):
        print('Date is 3 Weeks out, Setting to 3 Monday from now')
        start_date = start_date + timedelta(21)    
        
    elif(start_date_alt == 'Next_4_Week'):
        print('Date is 4 Weeks out, Setting to 4 Monday from now')
        start_date = start_date + timedelta(28)         

    elif(start_date_alt == 'Current_Week'):
        print('Date is 0 Week out, Setting to this week')
        start_date = start_date + timedelta(0)  
        
    elif(start_date_alt == '1_WeekAgo'):
        print('Date is 1 Week out, Setting to 1 week ago')
        start_date = start_date - timedelta(7)  
    
    elif(start_date_alt == '2_WeekAgo'):
        print('Date is 2 Weeks out, Setting to 2 week ago')
        start_date = start_date - timedelta(14)      
        
    elif(start_date_alt == '3_WeekAgo'):
        print('Date is 3 Weeks out, Setting to 3 week ago')
        start_date = start_date - timedelta(21)    
    
    else:
        print('Date is Current Week')


end_date = start_date + timedelta(7)
    
    
base = start_date
date_list = [base + timedelta(days=x) for x in range(7)]          
print ("Week start: %s   Week end: %s" % (start_date, end_date))

if(clientCode == 'ALL'):
    print("Running for All clients")
else:
    print("Running for client: ", clientCode)
    
print('-----------------------------------------------')


In [ ]:
focus_db = get_db_SS_focus()
report_RO_DB = Reporting_Read_Only()

if(clientCode == 'ALL'):
    sql="""get your data"""
    sql_pd1 = pd.read_sql(sql, focus_db)
    
    
else:
    sql="""get your data
        """   
    sql_pd1 = pd.read_sql(sql, focus_db)

    sql_pd4 = """  
   get your data
    """
    sql_pd4 = pd.read_sql(sql_pd4, report_RO_DB)
    sql_pd1 = pd.merge(sql_pd1,sql_pd4,how='left',left_on='Department',right_on='Department')

    sql_pd1 = sql_pd1[sql_pd1['Client'] == clientCode].reset_index(drop=True)
    sql_pd1 = sql_pd1[['MemberID', 'Name', 'Department', 'Schedule']]
    sql_pd1 = sql_pd1[sql_pd1['Schedule'].notnull()].reset_index(drop=True)

In [ ]:
schedule_df = sql_pd1
# output looks like this
# MemberID Name Department Schedule
# 1 a x {"Monday":[["08:00","17:00"]],"Tuesday":[["08:00","17:00"]],"Wednesday":[["08:00","17:00"]],"Thursday":[["08:00","17:00"]],"Friday":[["08:00","17:00"]]}
# 2 b y {"Monday":[["07:00","15:30"]],"Tuesday":[["07:00","15:30"]],"Wednesday":[["07:00","15:30"]],"Thursday":[["07:00","15:30"]],"Friday":[["07:00","15:30"]]}


In [ ]:
def convert_time_to_datetime(date_str, time_str):
    # Handle '24:00' by converting it to '00:00' of the next day
    if time_str == '24:00':
        time_str = '00:00'
        date_str = (datetime.strptime(date_str, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
    
    try:
        return datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
    except ValueError as e:
        print(f"Error converting time: {e}")
        return None

def calculate_shift_metrics(member_id,Name,Department, start_date, day_name, shift):
    start_time, end_time = shift
    start_datetime = convert_time_to_datetime(start_date, start_time)
    end_datetime = convert_time_to_datetime(start_date, end_time)
    
    scheduled_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    
    start_seconds = (start_datetime - scheduled_datetime).total_seconds()
    end_seconds = (end_datetime - scheduled_datetime).total_seconds()
    
    if start_datetime is None or end_datetime is None:
        return None

    if end_datetime < start_datetime:
        # Handle overnight shift
        end_datetime += timedelta(days=1)
    
    shift_length = (end_datetime - start_datetime).total_seconds()
    shift_minutes = shift_length / 60
    
    return {
        'MemberID': member_id,
        'Name': Name,  
        'Department': Department, 
        'Scheduled_Date': start_date,
        'Scheduled_Day': day_name,
        'Start_Time': start_time,
        'End_Time': end_time,
        'Start_DateTime': start_datetime,
        'End_DateTime': end_datetime,
        'Shift_Length': shift_length,
        'Start': start_seconds,
        'End': end_seconds,
        'ShiftMinutes': shift_minutes
    }

def find_monday(date):
    """Find the Monday of the week for a given date"""
    return date - timedelta(days=date.weekday())

def generate_metrics(schedule_df, reference_date):
    all_metrics = []
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday","Saturday","Sunday"]
    
    # Find Monday of the week for the given reference date
    start_of_week = find_monday(reference_date)
    
    # Ensure Schedule column is in dictionary format (if it's already a dictionary, no need to convert)
    def ensure_dict(schedule):
        if isinstance(schedule, str):
            try:
                return json.loads(schedule)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON string: {schedule}. Exception: {e}")
                return {}
        elif isinstance(schedule, dict):
            return schedule
        else:
            print(f"Unexpected format for schedule: {schedule}")
            return {}
    
    schedule_df['Schedule'] = schedule_df['Schedule'].apply(ensure_dict)
    
    for _, row in schedule_df.iterrows():
        member_id = row['MemberID']
        Name = row['Name']
        Department = row['Department']

        weekly_schedule = row['Schedule']
        
        # Debug: Check the type of weekly_schedule
        if not isinstance(weekly_schedule, dict):
            print(f"Error: weekly_schedule is not a dictionary. It is: {type(weekly_schedule)}")
            continue
        
        for day_index, day_name in enumerate(days_of_week):
            if day_name in weekly_schedule:
                shifts = weekly_schedule[day_name]
                
                # Debug: Check the type of shifts
                if not isinstance(shifts, list):
                    print(f"Error: Shifts for {day_name} are not a list. They are: {type(shifts)}")
                    continue
                
                # Calculate the date for this day
                date_str = (start_of_week + timedelta(days=day_index)).strftime("%Y-%m-%d")
                for shift in shifts:
                    if not isinstance(shift, list) or len(shift) != 2:
                        print(f"Error: Invalid shift format for {day_name}. Shift: {shift}")
                        continue
                        
                    metrics = calculate_shift_metrics(member_id,Name,Department, date_str, day_name, shift)
                    if metrics:  # Ensure metrics are not None
                        all_metrics.append(metrics)
    
    return pd.DataFrame(all_metrics)

reference_date = start_date 

# Generate metrics
metrics_df = generate_metrics(schedule_df, reference_date)

metrics_df

In [ ]:
MemberID	Name	Department	Scheduled_Date	Scheduled_Day	Start_Time	End_Time	Start_DateTime	End_DateTime	Shift_Length	Start	End	ShiftMinutes
1	a x		2025-04-21	Monday	08:00	17:00	2025-04-21 08:00:00	2025-04-21 17:00:00	32400.0	28800.0	61200.0	540.0
1	a y		2025-04-22	Tuesday	08:00	17:00	2025-04-22 08:00:00	2025-04-22 17:00:00	32400.0	28800.0	61200.0	540.0

In [ ]:
df = metrics_df

# Convert to datetime
df['Start_DateTime'] = pd.to_datetime(df['Start_DateTime'])
df['End_DateTime'] = pd.to_datetime(df['End_DateTime'])
df['Scheduled_DateTime'] = pd.to_datetime(df['Scheduled_Date'])

# Sort by MemberID and Start_DateTime
df.sort_values(by=['MemberID', 'Start_DateTime'], inplace=True)

# Initialize lists to store results
consolidated_shifts = []

# Process each group of MemberID
for member_id, group in df.groupby('MemberID'):
    # Variables to store previous row information
    prev_row = None

    for idx, row in group.iterrows():
        if prev_row is not None:
            # If previous shift ends at 24:00 and current shift starts at 00:00, combine them
            if prev_row['End_Time'] == '24:00' and row['Start_Time'] == '00:00':
                consolidated_shifts.append({
                    'MemberID': member_id,
                    'Name': prev_row['Name'],
                    'Department': prev_row['Department'],
                    'Scheduled_Date': prev_row['Scheduled_Date'],
                    'Scheduled_Day': prev_row['Scheduled_Day'],
                    'Start_DateTime': prev_row['Start_DateTime'],
                    'End_DateTime': row['End_DateTime'],
                    'Shift_Length': (row['End_DateTime'] - prev_row['Start_DateTime']).total_seconds(),
                    'ShiftMinutes': (row['End_DateTime'] - prev_row['Start_DateTime']).total_seconds() / 60,
                    'Start': (prev_row['Start_DateTime'] - prev_row['Scheduled_DateTime']).total_seconds(),
                    'End': (row['End_DateTime'] - prev_row['Scheduled_DateTime']).total_seconds(),
                    'Overnight': "Y"
                })
                # Skip the current row as it's merged
                prev_row = None
            else:
                # Check if the shift is overnight
                overnight = "Y" if prev_row['Start_DateTime'].date() != prev_row['End_DateTime'].date() else "N"
                
                consolidated_shifts.append({
                    'MemberID': member_id,
                    'Name': prev_row['Name'],
                    'Department': prev_row['Department'],
                    'Scheduled_Date': prev_row['Scheduled_Date'],
                    'Scheduled_Day': prev_row['Scheduled_Day'],
                    'Start_DateTime': prev_row['Start_DateTime'],
                    'End_DateTime': prev_row['End_DateTime'],
                    'Shift_Length': (prev_row['End_DateTime'] - prev_row['Start_DateTime']).total_seconds(),
                    'ShiftMinutes': (prev_row['End_DateTime'] - prev_row['Start_DateTime']).total_seconds() / 60,
                    'Start': (prev_row['Start_DateTime'] - prev_row['Scheduled_DateTime']).total_seconds(),
                    'End': (prev_row['End_DateTime'] - prev_row['Scheduled_DateTime']).total_seconds(),
                    'Overnight': overnight
                })
                prev_row = row
        else:
            prev_row = row

    # Add the last row if it wasn't merged
    if prev_row is not None:
        # Check if the shift is overnight
        overnight = "Y" if prev_row['Start_DateTime'].date() != prev_row['End_DateTime'].date() else "N"

        consolidated_shifts.append({
            'MemberID': member_id,
            'Name': prev_row['Name'],
            'Department': prev_row['Department'],
            'Scheduled_Date': prev_row['Scheduled_Date'],
            'Scheduled_Day': prev_row['Scheduled_Day'],
            'Start_DateTime': prev_row['Start_DateTime'],
            'End_DateTime': prev_row['End_DateTime'],
            'Shift_Length': (prev_row['End_DateTime'] - prev_row['Start_DateTime']).total_seconds(),
            'ShiftMinutes': (prev_row['End_DateTime'] - prev_row['Start_DateTime']).total_seconds() / 60,
            'Start': (prev_row['Start_DateTime'] - prev_row['Scheduled_DateTime']).total_seconds(),
            'End': (prev_row['End_DateTime'] - prev_row['Scheduled_DateTime']).total_seconds(),
            'Overnight': overnight
        })

consolidated_df = pd.DataFrame(consolidated_shifts)
consolidated_df


# Adjustment for following week schedule.

In [ ]:
df = consolidated_df
# Convert Start_DateTime and End_DateTime to datetime format
df['Start_DateTime'] = pd.to_datetime(df['Start_DateTime'])
df['End_DateTime'] = pd.to_datetime(df['End_DateTime'])

# Function to adjust the Sunday shift ending on Monday
def adjust_sunday_shift(df):
    # Identify the Sunday shift that continues into Monday
    sunday_shift = df[(df['Scheduled_Day'] == 'Sunday') & (df['End_DateTime'].dt.weekday == 0)]
    
    # Loop through each Sunday shift
    for idx, shift in sunday_shift.iterrows():
        member_id = shift['MemberID']
        
        # Find the corresponding Monday shift with a start time of 00:00:00 for the same week
        monday_shift = df[(df['MemberID'] == member_id) & 
                          (df['Scheduled_Day'] == 'Monday') & 
                          (df['Start_DateTime'].dt.time == datetime.strptime("00:00:00", "%H:%M:%S").time())]
        print(monday_shift)
        
        if not monday_shift.empty:
            # Get the time of the Monday shift
            monday_start_time = monday_shift.iloc[0]['End_DateTime'].time()
            print(monday_start_time)

            
            # Update the Sunday shift's end time to the same date but the time of the Monday shift
            new_end_datetime = shift['End_DateTime'].replace(hour=monday_start_time.hour, 
                                                             minute=monday_start_time.minute, 
                                                             second=monday_start_time.second, 
                                                             microsecond=0)
            print(new_end_datetime)

            # Adjust the date to the next day if the updated end time is before or equal to the original end time
#             if new_end_datetime <= shift['End_DateTime']:
#                 new_end_datetime = new_end_datetime + timedelta(days=1)
            
            # Set the updated end datetime
            df.at[idx, 'End_DateTime'] = new_end_datetime
            
            # Recalculate Shift_Length and ShiftMinutes
            df.at[idx, 'Shift_Length'] = (new_end_datetime - shift['Start_DateTime']).total_seconds()
            df.at[idx, 'ShiftMinutes'] = df.at[idx, 'Shift_Length'] / 60
            
    return df

# Apply the adjustment
df = adjust_sunday_shift(df)

# Display the updated DataFrame
consolidated_df = df
consolidated_df

In [ ]:
# dont use agents with PTO for scheduled day
pto_Scheduled = """ get your pto

"""
pto_Scheduled = pto_Scheduled % (start_date)    
pto_Scheduled = pd.read_sql(pto_Scheduled, focus_db)

pto_Scheduled['MemberID'] = pto_Scheduled['MemberID'].astype(int)
pto_Scheduled['Scheduled_Date'] = pd.to_datetime(pto_Scheduled['Scheduled_Date']).dt.date
consolidated_df['MemberID'] = consolidated_df['MemberID'].astype(int)
consolidated_df['Scheduled_Date'] = pd.to_datetime(consolidated_df['Scheduled_Date']).dt.date

PTO_Check = consolidated_df.merge(pto_Scheduled, on=['MemberID','Scheduled_Date'], how='left')
PTO_Check.replace(np.nan, 0, inplace=True)

PTO_Check['PTO Scheduled'] = np.where(PTO_Check['PTO Mins'] > 0 ,'Y','N')
Schedule_Agents = PTO_Check[PTO_Check['PTO Scheduled']=='N']

PTO_ignore_schedule = PTO_Check[PTO_Check['PTO Scheduled']=='Y']
print(PTO_ignore_schedule)

Schedule_Agents

In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
focus_db = get_db_SS_focus()

Insert_Start_SQL = 'LunchStart'
Insert_Stop_SQL = 'LunchStop'
Insert_Seconds_SQL = 'LunchSeconds'

Insert_Reporting_Start = 'Lunch_start_window'
Insert_Reporting_End = 'Lunch_end_window'
Insert_Repoting_OnOff = 'Lunch'    


report_RO_DB = Reporting_Read_Only()
shift_Reporting_Sql = """
select Shift_Length_hrs*60 as 'ShiftMinutes', 
48 as Time_start_window, %s as Time_end_window,
%s as 'Lunch_Option',
(case when %s = 'Y' then 30 else 0 end) as 'Expected_Lunch_Time' 
from workforce.wfm_break_lunch wbl """

shift_Reporting_Sql = shift_Reporting_Sql % (Insert_Reporting_End,Insert_Repoting_OnOff,Insert_Repoting_OnOff)


lunch_pd = pd.read_sql(shift_Reporting_Sql, report_RO_DB) 
lunch_pd
# Merge lunch data with Schedule_Agents shifts
agents_lunch_assign = pd.merge(Schedule_Agents, lunch_pd, how='left', on='ShiftMinutes')
agents_lunch_assign['Time_start_window'].fillna(0, inplace=True)
agents_lunch_assign['Time_end_window'].fillna(0, inplace=True)
agents_lunch_assign['Lunch_Option'].fillna('N', inplace=True)


#Override preset lunch duration
report_RO_DB = Reporting_Read_Only()
Block_Sch_Override = """
select MemberID, Lunch_Duration, Lunch_Duration_Min 
from workforce.blocksch_member_settings 
where Lunch_Duration = 'Y'
"""
Block_Sch_Override = pd.read_sql(Block_Sch_Override, report_RO_DB)
agents_lunch_assign = pd.merge(agents_lunch_assign, Block_Sch_Override, on=['MemberID'],how='left')
agents_lunch_assign['Lunch_Duration'].replace(np.nan,"N",inplace=True)
agents_lunch_assign['Expected_Lunch_Time'] = np.where(agents_lunch_assign['Lunch_Duration'] == 'Y', agents_lunch_assign['Lunch_Duration_Min'],agents_lunch_assign['Expected_Lunch_Time'])
agents_lunch_assign = agents_lunch_assign.fillna(0)
agents_lunch_assign['Expected_Lunch_Time']  = agents_lunch_assign['Expected_Lunch_Time'].astype(int)

agents_lunch_assign = agents_lunch_assign.drop(['Lunch_Duration', 'Lunch_Duration_Min'], axis=1)


# Maintain a list of recently assigned lunch start times
last_parameter_list = []
max_round_robins = 3  # Maximum number of recent values to remember

def generate_lunch_start(start_dt, end_dt, start_window, end_window, lunch_option, overnight_flag,lunch_duration,memberid):
    if lunch_option == 'N':
        return None, None

    shift_duration = (end_dt - start_dt).total_seconds()
    start_seconds = start_window * (shift_duration / 100)
    end_seconds = end_window * (shift_duration / 100)

    start_time = start_dt + timedelta(seconds=start_seconds)
    end_time = start_dt + timedelta(seconds=end_seconds)

    # Ensure that lunch start time is within start and end window
    duration_seconds = (lunch_duration*60)

    if((end_seconds - duration_seconds - start_seconds) <0):
        start_seconds = start_seconds - 900
        
        if((end_seconds - duration_seconds - start_seconds) <0):
            end_seconds = end_seconds + 900

    lunch_start_seconds = np.random.randint(start_seconds, end_seconds - duration_seconds)

    lunch_start = start_dt + timedelta(seconds=lunch_start_seconds)
    lunch_end = lunch_start + timedelta(minutes=lunch_duration)  # Ensuring exact 30-minute duration

    # Adjust if the 30-minute lunch exceeds the end of the window
    if lunch_end > end_time:
        lunch_start = end_time - timedelta(minutes=lunch_duration)
        lunch_end = end_time

    # Collision handling
    lunch_start_seconds = int((lunch_start - start_dt).total_seconds())
    while lunch_start_seconds in last_parameter_list:
        lunch_start_seconds = (lunch_start_seconds + 900) % 86400
        lunch_start = start_dt + timedelta(seconds=lunch_start_seconds)
        lunch_end = lunch_start + timedelta(minutes=lunch_duration)
        if lunch_end > end_time:
            break

    # Append to the list and maintain its length
    last_parameter_list.append(lunch_start_seconds)
    if len(last_parameter_list) > max_round_robins:
        last_parameter_list.pop(0)
#     print (lunch_start,"Lunchstart")

    lunch_start = round_to_nearest_interval(lunch_start,overnight_flag)
    lunch_end = lunch_start + timedelta(minutes=lunch_duration)

    # Final boundary check to ensure times remain within start and end window
    if lunch_start < start_time:
        lunch_start = start_time
        lunch_end = lunch_start + timedelta(minutes=lunch_duration)
    if lunch_end > end_time:
        lunch_end = end_time
        lunch_start = lunch_end - timedelta(minutes=lunch_duration)
        
    lunch_start = round_to_nearest_interval(lunch_start,overnight_flag)
    lunch_end = lunch_start + timedelta(minutes=lunch_duration)

    return lunch_start, lunch_end


def round_to_nearest_interval(dt, overnight_flag):
    if overnight_flag == 'Y':
        interval = 30  # Round to nearest 30 minutes
    else:
        interval = 15  # Round to nearest 15 minutes
    

    # Calculate the nearest 15-minute interval
    nearest_multiple_of_15 = round(dt.minute / 15) * 15
    
    # Adjust the datetime based on the calculated interval
    rounded_dt = dt.replace(minute=nearest_multiple_of_15 % 60, second=0, microsecond=0)
    
    # Handle overflow to the next hour if rounding to 60 minutes
    if nearest_multiple_of_15 == 60:
        rounded_dt += timedelta(hours=1)
    
    return rounded_dt


# Define function to process each row
def process_row_for_lunch(row):
    lunch_start, lunch_end = generate_lunch_start(
        row['Start_DateTime'], row['End_DateTime'],
        row['Time_start_window'], row['Time_end_window'],
        row['Lunch_Option'], row['Overnight'],row['Expected_Lunch_Time'],
        row['MemberID']
    )
    
    # Calculate Lunch_Seconds if both start and end times are not None
    lunch_seconds = (lunch_end - lunch_start).total_seconds() if lunch_start and lunch_end else 0
    
    return pd.Series({
        'MemberID': row['MemberID'],
        'Name': row['Name'],
        'Department': row['Department'],
        'Scheduled_Date': row['Scheduled_Date'],
        'Scheduled_Day': row['Scheduled_Day'],
        'Start_DateTime': row['Start_DateTime'],
        'End_DateTime': row['End_DateTime'],
        'Shift_Length': row['Shift_Length'],
        'ShiftMinutes': row['ShiftMinutes'],
        'Lunch_Option': row['Lunch_Option'],
        'Lunch_Start_DateTime': lunch_start,
        'Lunch_End_DateTime': lunch_end,
        'Lunch_Seconds': lunch_seconds,
        'Overnight': row['Overnight'],
    })

# Apply the function to each row in the DataFrame
lunch_assignments_df = agents_lunch_assign.apply(process_row_for_lunch, axis=1)

# Convert Scheduled_Date column to datetime if it's not already
lunch_assignments_df['Scheduled_DateTime'] = pd.to_datetime(lunch_assignments_df['Scheduled_Date'])

# Calculate Start and End in seconds since midnight
lunch_assignments_df['Start'] = (lunch_assignments_df['Start_DateTime'] - lunch_assignments_df['Scheduled_DateTime']).dt.total_seconds()
lunch_assignments_df['End'] = (lunch_assignments_df['End_DateTime'] - lunch_assignments_df['Scheduled_DateTime']).dt.total_seconds()

# Display the resulting DataFrame
lunch_assignments_df


In [ ]:
Insert_Start_SQL = 'BreakAStart'
Insert_Stop_SQL = 'BreakAStop'
Insert_Seconds_SQL = 'BreakASeconds'

Insert_Reporting_Start = 'BreakA_Start'
Insert_Reporting_End = 'BreakA_End'
Insert_Repoting_OnOff = 'BreakA'    


report_RO_DB = Reporting_Read_Only()
shift_Reporting_Sql = """
select Shift_Length_hrs*60 as 'ShiftMinutes', 
%s as Time_start_window, %s as Time_end_window,
%s as 'BreakA_Option',
(case when %s = 'Y' then 15 else 0 end) as 'Expected_BreakA_Time' 
from workforce.wfm_break_lunch wbl """

shift_Reporting_Sql = shift_Reporting_Sql % (Insert_Reporting_Start,Insert_Reporting_End,Insert_Repoting_OnOff,Insert_Repoting_OnOff)

BreakA_pd = pd.read_sql(shift_Reporting_Sql, report_RO_DB) 

Agents_BreakA_Assign = pd.merge(lunch_assignments_df,BreakA_pd,how='left',left_on='ShiftMinutes',right_on='ShiftMinutes')
Agents_BreakA_Assign['Time_start_window'].fillna(0, inplace=True)
Agents_BreakA_Assign['Time_end_window'].fillna(0, inplace=True)
Agents_BreakA_Assign['BreakA_Option'].fillna('N', inplace=True)


#Override preset Break  duration
report_RO_DB = Reporting_Read_Only()
Block_Sch_Override = """
select MemberID, breakA_Duration, breakA_Duration_Min 
from workforce.blocksch_member_settings 
where breakA_Duration = 'Y'
"""
Block_Sch_Override = pd.read_sql(Block_Sch_Override, report_RO_DB)
Agents_BreakA_Assign = pd.merge(Agents_BreakA_Assign, Block_Sch_Override, on=['MemberID'],how='left')
Agents_BreakA_Assign['breakA_Duration'].replace(np.nan,"N",inplace=True)
Agents_BreakA_Assign['Expected_BreakA_Time'] = np.where(Agents_BreakA_Assign['breakA_Duration'] == 'Y', Agents_BreakA_Assign['breakA_Duration_Min'],Agents_BreakA_Assign['Expected_BreakA_Time'])
Agents_BreakA_Assign = Agents_BreakA_Assign.drop(['breakA_Duration', 'breakA_Duration_Min'], axis=1)



# Maintain a list of recently assigned BreakA start times
last_breakA_parameter_list = []
max_breakA_round_robins = 3  # Maximum number of recent values to remember for BreakA

def generate_BreakA_start(start_dt, end_dt, start_window, end_window, break_option,breakA_duration):
    if break_option == 'N':
        return None, None

    # Calculate the shift duration in seconds
    shift_duration = (end_dt - start_dt).total_seconds()

    duration_seconds = (breakA_duration*60)
    
    # Calculate start and end of the window in seconds
    start_seconds = start_window * (shift_duration / 100)
    end_seconds = end_window * (shift_duration / 100)
    print(start_dt)
    print(end_dt)

    print(start_seconds)
    print(end_seconds)


    # Calculate the start and end time of the window
    start_time = start_dt + timedelta(seconds=start_seconds)
    end_time = start_dt + timedelta(seconds=end_seconds)
    
    # Generate a random BreakA start time within the window
    breakA_start = start_time + timedelta(seconds=np.random.randint(0, int(end_seconds - start_seconds)))
    
    # Adjust for collisions
    breakA_start_seconds = breakA_start.hour * 3600 + breakA_start.minute * 60
    while breakA_start_seconds in last_breakA_parameter_list:
        breakA_start_seconds = (breakA_start_seconds + 900) % 86400  # Shift by 15 minutes
        breakA_start = breakA_start.replace(hour=0, minute=0, second=0) + timedelta(seconds=breakA_start_seconds)
    
    # Update the list of recent values
    last_breakA_parameter_list.append(breakA_start_seconds)
    if len(last_breakA_parameter_list) > max_breakA_round_robins:
        last_breakA_parameter_list.pop(0)
    
    # Round to nearest 15 minutes
    breakA_start = round_to_nearest_15_minutes(breakA_start)
    breakA_end = breakA_start + timedelta(minutes=breakA_duration)
    
    # Adjust if BreakA crosses midnight or end of the shift
    if breakA_end > end_dt:
        breakA_end = end_dt
        breakA_start = breakA_end - timedelta(minutes=breakA_duration)

    return breakA_start, breakA_end

# Define round_to_nearest_15_minutes function
def round_to_nearest_15_minutes(dt):
    minute = (dt.minute // 15) * 15
    return dt.replace(minute=minute, second=0, microsecond=0)

# Apply function to assign BreakA
def process_row_for_breakA(row):
    breakA_start, breakA_end = generate_BreakA_start(
        row['Start_DateTime'], row['End_DateTime'],
        row['Time_start_window'], row['Time_end_window'],
        row['BreakA_Option'] ,row['Expected_BreakA_Time']
    )
    
    # Calculate BreakA_Seconds if both start and end times are not None
    breakA_seconds = (breakA_end - breakA_start).total_seconds() if breakA_start and breakA_end else 0
    
    return pd.Series({
        'MemberID': row['MemberID'],
        'Name': row['Name'],
        'Department': row['Department'],
        'Scheduled_Date': row['Scheduled_Date'],
        'Scheduled_Day': row['Scheduled_Day'],
        'Start_DateTime': row['Start_DateTime'],
        'End_DateTime': row['End_DateTime'],
        'Shift_Length': row['Shift_Length'],
        'ShiftMinutes': row['ShiftMinutes'],
        'Lunch_Option':row['Lunch_Option'],

        'Lunch_Start_DateTime': row['Lunch_Start_DateTime'],
        'Lunch_End_DateTime': row['Lunch_End_DateTime'],
        'Lunch_Seconds': row['Lunch_Seconds'],
        'BreakA_Option':row['BreakA_Option'],

        'BreakA_Start_DateTime': breakA_start,
        'BreakA_End_DateTime': breakA_end,
        'BreakA_Seconds': breakA_seconds,
        'Overnight' :row['Overnight'],

    })

# Apply the function to each row in the DataFrame
BreakA_assignments_df = Agents_BreakA_Assign.apply(process_row_for_breakA, axis=1)

# Convert Scheduled_Date column to datetime if it's not already
BreakA_assignments_df['Scheduled_DateTime'] = pd.to_datetime(BreakA_assignments_df['Scheduled_Date'])

# Calculate Start and End in seconds since midnight
BreakA_assignments_df['Start'] = (BreakA_assignments_df['Start_DateTime'] - BreakA_assignments_df['Scheduled_DateTime']).dt.total_seconds()
BreakA_assignments_df['End'] = (BreakA_assignments_df['End_DateTime'] - BreakA_assignments_df['Scheduled_DateTime']).dt.total_seconds()

# Display the resulting DataFrame
BreakA_assignments_df

In [ ]:
Insert_Start_SQL = 'BreakBStart'
Insert_Stop_SQL = 'BreakBStop'
Insert_Seconds_SQL = 'BreakBSeconds'

Insert_Reporting_Start = 'BreakB_Start'
Insert_Reporting_End = 'BreakB_End'
Insert_Repoting_OnOff = 'BreakB'    


report_RO_DB = Reporting_Read_Only()
shift_Reporting_Sql = """
select Shift_Length_hrs*60 as 'ShiftMinutes', 
%s as Time_start_window, %s as Time_end_window,
%s as 'BreakB_Option',
(case when %s = 'Y' then 15 else 0 end) as 'Expected_BreakB_Time' 
from workforce.wfm_break_lunch wbl """

shift_Reporting_Sql = shift_Reporting_Sql % (Insert_Reporting_Start,Insert_Reporting_End,Insert_Repoting_OnOff,Insert_Repoting_OnOff)

BreakB_pd = pd.read_sql(shift_Reporting_Sql, report_RO_DB) 

Agents_BreakB_Assign = pd.merge(BreakA_assignments_df,BreakB_pd,how='left',left_on='ShiftMinutes',right_on='ShiftMinutes')
# Agents_BreakB_Assign.replace(np.nan,0,inplace=True)
Agents_BreakB_Assign['Time_start_window'].fillna(0, inplace=True)
Agents_BreakB_Assign['Time_end_window'].fillna(0, inplace=True)
Agents_BreakB_Assign['BreakB_Option'].fillna('N', inplace=True)


#Override preset Break  duration
report_RO_DB = Reporting_Read_Only()
Block_Sch_Override = """
select MemberID, breakB_Duration, breakB_Duration_Min 
from workforce.blocksch_member_settings 
where breakB_Duration = 'Y'
"""
Block_Sch_Override = pd.read_sql(Block_Sch_Override, report_RO_DB)
Agents_BreakB_Assign = pd.merge(Agents_BreakB_Assign, Block_Sch_Override, on=['MemberID'],how='left')
Agents_BreakB_Assign['breakB_Duration'].replace(np.nan,"N",inplace=True)
Agents_BreakB_Assign['Expected_BreakB_Time'] = np.where(Agents_BreakB_Assign['breakB_Duration'] == 'Y', Agents_BreakB_Assign['breakB_Duration_Min'],Agents_BreakB_Assign['Expected_BreakB_Time'])
Agents_BreakB_Assign = Agents_BreakB_Assign.drop(['breakB_Duration', 'breakB_Duration_Min'], axis=1)

# Maintain a list of recently assigned BreakB start times
last_breakB_parameter_list = []
max_breakB_round_robins = 3  # Maximum number of recent values to remember for BreakB

def generate_breakB_start(start_dt, end_dt, start_window, end_window, breakB_option,breakB_duration):
    if breakB_option == 'N':
        return None, None

    duration_seconds = (breakB_duration*60)

    shift_duration = (end_dt - start_dt).total_seconds()
    start_seconds = start_window * (shift_duration / 100)
    end_seconds = end_window * (shift_duration / 100)
    
    start_time = start_dt + timedelta(seconds=start_seconds)
    end_time = start_dt + timedelta(seconds=end_seconds)
    
    # Generate a random BreakB start time
    breakB_start = start_time + timedelta(seconds=np.random.randint(0, int(end_seconds - start_seconds)))
    
    # Adjust for collisions
    breakB_start_seconds = breakB_start.hour * 3600 + breakB_start.minute * 60
    while breakB_start_seconds in last_breakB_parameter_list:
        breakB_start_seconds = (breakB_start_seconds + 900) % 86400  # Shift by 15 minutes
        breakB_start = breakB_start.replace(hour=0, minute=0, second=0) + timedelta(seconds=breakB_start_seconds)
    
    last_breakB_parameter_list.append(breakB_start_seconds)
    if len(last_breakB_parameter_list) > max_breakB_round_robins:
        last_breakB_parameter_list.pop(0)
    
    breakB_start = round_to_nearest_15_minutes(breakB_start)  # Round to nearest 15 minutes
    breakB_end = breakB_start + timedelta(minutes=breakB_duration)
    
    # Adjust if BreakB crosses midnight or end of the shift
    if breakB_end > end_dt:
        breakB_end = end_dt
        breakB_start = breakB_end - timedelta(minutes=breakB_duration)

    return breakB_start, breakB_end

# Function to round BreakB start time to the nearest 15 minutes
def round_to_nearest_15_minutes(dt):
    minute = (dt.minute // 15) * 15
    return dt.replace(minute=minute, second=0, microsecond=0)

# Apply function to assign BreakB
breakB_assignments = []

for index, row in Agents_BreakB_Assign.iterrows():
    breakB_start, breakB_end = generate_breakB_start(
        row['Start_DateTime'], row['End_DateTime'],
        row['Time_start_window'], row['Time_end_window'],
        row['BreakB_Option'],row['Expected_BreakB_Time']  # Assuming there's a column for BreakB options
    )
    
    breakB_assignments.append({
        'MemberID': row['MemberID'],
        'Name': row['Name'],
        'Department': row['Department'],
        'Scheduled_Date': row['Scheduled_Date'],
        'Scheduled_Day': row['Scheduled_Day'],
        'Start_DateTime': row['Start_DateTime'],
        'End_DateTime': row['End_DateTime'],
        'Shift_Length': row['Shift_Length'],
        'ShiftMinutes': row['ShiftMinutes'],
        'Lunch_Option':row['Lunch_Option'],
        'Lunch_Start_DateTime' : row['Lunch_Start_DateTime'],
        'Lunch_End_DateTime' : row['Lunch_End_DateTime'],
        'Lunch_Seconds':row['Lunch_Seconds'],
        'BreakA_Option':row['BreakA_Option'],
        'BreakA_Start_DateTime':row['BreakA_Start_DateTime'],
        'BreakA_End_DateTime':row['BreakA_End_DateTime'],
        'BreakA_Seconds':row['BreakA_Seconds'],   
        'BreakB_Option':row['BreakB_Option'],
        'BreakB_Start_DateTime': breakB_start,
        'BreakB_End_DateTime': breakB_end,
        'BreakB_Seconds': (breakB_end - breakB_start).total_seconds() if breakB_start and breakB_end else 0,
        'Overnight' :row['Overnight'],

    })

# Convert to DataFrame
breakB_assignments_df = pd.DataFrame(breakB_assignments)
breakB_assignments_df['Scheduled_DateTime'] = pd.to_datetime(breakB_assignments_df['Scheduled_Date'])

breakB_assignments_df['Start'] = (breakB_assignments_df['Start_DateTime'] - breakB_assignments_df['Scheduled_DateTime']).dt.total_seconds()
breakB_assignments_df['End'] = (breakB_assignments_df['End_DateTime'] - breakB_assignments_df['Scheduled_DateTime']).dt.total_seconds()

breakB_assignments_df


In [ ]:
All_assigned_shifts = breakB_assignments_df
All_assigned_shifts

In [ ]:
# Convert datetime columns
datetime_cols = ['Start_DateTime', 'End_DateTime', 'Lunch_Start_DateTime', 'Lunch_End_DateTime', 
                  'BreakA_Start_DateTime', 'BreakA_End_DateTime', 'BreakB_Start_DateTime', 'BreakB_End_DateTime']
All_assigned_shifts[datetime_cols] = All_assigned_shifts[datetime_cols].apply(pd.to_datetime, errors='coerce')

# Initialize list for split data
split_data = []

# Split shifts
for idx, row in All_assigned_shifts.iterrows():
    if row['Overnight'] == 'Y':
        print(idx)
        print(row)
        # Calculate the split time (midnight)
        split_time = (row['Start_DateTime'] + pd.DateOffset(days=1)).normalize()

        
        # First part of the shift
        first_part = row.copy()
        first_part['End_DateTime'] = split_time
        first_part['Shift_Length'] = (split_time - row['Start_DateTime']).total_seconds()
        first_part['ShiftMinutes'] = first_part['Shift_Length'] / 60
        first_part['Start_DateTime'] = row['Start_DateTime']
        first_part['Scheduled_Date'] = row['Scheduled_Date']
        first_part['Scheduled_Day'] = row['Scheduled_Day']
        
        # Second part of the shift
        second_part = row.copy()
        second_part['Start_DateTime'] = split_time
        second_part['End_DateTime'] = row['End_DateTime']
        second_part['Shift_Length'] = (row['End_DateTime'] - split_time).total_seconds()
        second_part['ShiftMinutes'] = second_part['Shift_Length'] / 60
        second_part['Scheduled_Date'] = (pd.Timestamp(row['Scheduled_Date']) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        second_part['Scheduled_Day'] = (pd.Timestamp(row['Scheduled_Date']) + pd.DateOffset(days=1)).day_name()

        # Allocate lunch and breaks
        for col in ['Lunch', 'BreakA', 'BreakB']:
            start_col = f'{col}_Start_DateTime'
            end_col = f'{col}_End_DateTime'
            start_dates = row[start_col].date() if not pd.isna(row[start_col]) else None
            end_dates = row[end_col].date() if not pd.isna(row[end_col]) else None

            # Allocate lunch and breaks for the first part
            if start_dates == row['Start_DateTime'].date() and end_dates and end_dates <= split_time.date():
                first_part[start_col] = row[start_col]
                first_part[end_col] = row[end_col]
                first_part[f'{col}_Seconds'] = row[f'{col}_Seconds']
            else:
                first_part[start_col] = None
                first_part[end_col] = None
                first_part[f'{col}_Seconds'] = 0

            # Allocate lunch and breaks for the second part
            if start_dates == split_time.date():
                second_part[start_col] = row[start_col] if not pd.isna(row[start_col]) else None
                second_part[end_col] = row[end_col] if not pd.isna(row[end_col]) else None
                second_part[f'{col}_Seconds'] = row[f'{col}_Seconds']
            else:
                second_part[start_col] = None
                second_part[end_col] = None
                second_part[f'{col}_Seconds'] = 0

        split_data.append(first_part)
        split_data.append(second_part)
    else:
        split_data.append(row)

# Create new DataFrame
split_df = pd.DataFrame(split_data)

# Reset index
split_df.reset_index(drop=True, inplace=True)
split_df = split_df[split_df['Shift_Length']!=0]
split_df

In [ ]:
Training_Att_sql="""
SELECT MemberID, ProjectID, TypeID FROM SS_focus.MemberTraining mt 
where Ranking = 1
"""
Training_Att_sql = pd.read_sql(Training_Att_sql, focus_db)

Agent_Data_Only_Final = pd.merge(split_df,Training_Att_sql,on=['MemberID'],how='left')
Agent_Data_Only_Final = Agent_Data_Only_Final[Agent_Data_Only_Final['ProjectID'].notnull()]
Agent_Data_Only_Final

In [ ]:
# lunch_assignments_df.to_excel("lunch_assignments_df.xlsx")

# Clean up

In [ ]:
report_RO_DB = Reporting_Read_Only()
Block_Sch_Override = """
select * from workforce.blocksch_member_settings """
Block_Sch_Override = pd.read_sql(Block_Sch_Override, report_RO_DB)

WFM_Agent_Override = pd.merge(Agent_Data_Only_Final, Block_Sch_Override, on=['MemberID'],how='left')

WFM_Agent_Override['Override_breakA'].replace(np.nan,"N",inplace=True)
WFM_Agent_Override['Override_breakB'].replace(np.nan,"N",inplace=True)
WFM_Agent_Override['Override_Lunch'].replace(np.nan,"N",inplace=True)
WFM_Agent_Override['Override_FullSchedule'].replace(np.nan,"N",inplace=True)

WFM_Agent_Override['Lunch_Option'] = np.where(WFM_Agent_Override['Override_Lunch'] == 'Y', "N",WFM_Agent_Override['Lunch_Option'])
WFM_Agent_Override['Lunch_Seconds'] = np.where(WFM_Agent_Override['Override_Lunch'] == 'Y', 0,WFM_Agent_Override['Lunch_Seconds'])
WFM_Agent_Override['BreakA_Option'] = np.where(WFM_Agent_Override['Override_breakA'] == 'Y', "N",WFM_Agent_Override['BreakA_Option'])
WFM_Agent_Override['BreakA_Seconds'] = np.where(WFM_Agent_Override['Override_breakA'] == 'Y', 0,WFM_Agent_Override['BreakA_Seconds'])
WFM_Agent_Override['BreakB_Option'] = np.where(WFM_Agent_Override['Override_breakB'] == 'Y', "N",WFM_Agent_Override['BreakB_Option'])
WFM_Agent_Override['BreakB_Seconds'] = np.where(WFM_Agent_Override['Override_breakB'] == 'Y', 0,WFM_Agent_Override['BreakB_Seconds'])

WFM_Agent_Override = WFM_Agent_Override[WFM_Agent_Override['Override_FullSchedule'] != 'Y'].reset_index(drop=True)       


check_Already_Scheduled = """    SELECT 
ess.id as 'ShiftID',
m.id as 'MemberID', ShiftStart,ShiftStop
FROM SS_focus.EmployeeScheduleShift ess
join SS_focus.Member m on m.id = ess.MemberID 
where Date(ShiftStart) >= '%s' 
"""
check_Already_Scheduled = check_Already_Scheduled % (start_date)    
check_Already_Scheduled = pd.read_sql(check_Already_Scheduled, focus_db)
check_Already_Scheduled

In [ ]:
# Check Schedule and overlap

In [ ]:
df1 = WFM_Agent_Override
df2 = check_Already_Scheduled
# Convert relevant columns to datetime
df1['Start_DateTime'] = pd.to_datetime(df1['Start_DateTime'])
df1['End_DateTime'] = pd.to_datetime(df1['End_DateTime'])
df2['ShiftStart'] = pd.to_datetime(df2['ShiftStart'])
df2['ShiftStop'] = pd.to_datetime(df2['ShiftStop'])

# Calculate duration of shifts
df1['Duration'] = df1['End_DateTime'] - df1['Start_DateTime']
df2['Duration'] = df2['ShiftStop'] - df2['ShiftStart']

# Initialize a DataFrame to hold the updated df1
df1_updated = df1.copy()

# Process each MemberID separately
for member_id in df1_updated['MemberID'].unique():
#     print (member_id)
    # Filter shifts for this MemberID in both df1 and df2
    df1_member = df1_updated[df1_updated['MemberID'] == member_id]
    df2_member = df2[df2['MemberID'] == member_id]

    # Iterate over each shift in df1_member
    for idx1, row1 in df1_member.iterrows():
        start1 = row1['Start_DateTime']
        end1 = row1['End_DateTime']
        duration1 = row1['Duration']
        date1 = start1.date()  # Extract date part for checking daily shifts
        
        shift_id = None  # Initialize shift_id

        exact_match_found = False
        overlap_found = False
        
        for idx2, row2 in df2_member.iterrows():
            start2 = row2['ShiftStart']
            end2 = row2['ShiftStop']
            duration2 = row2['Duration']
            date2 = start2.date()  # Extract date part for checking daily shifts

            # Check if both shifts are on the same day
            if date1 == date2:
                # Check for exact match
                if start1 == start2 and end1 == end2 and duration1 == duration2:
                    exact_match_found = True
                    shift_id = row2['ShiftID']  # Assign ShiftID from df2
                    break  # Stop checking once an exact match is found
                
                # Check for overlap: (Shift1 starts before Shift2 ends) and (Shift1 ends after Shift2 starts)
                elif (start1 < end2) and (end1 > start2):
                    overlap_found = True
                    shift_id = row2['ShiftID']  # Assign ShiftID from df2 for overlapping shift

        # Update df1_updated with ShiftID based on match status
        if exact_match_found:
            df1_updated.at[idx1, 'ShiftID'] = shift_id  # Use the exact ShiftID from df2
        elif overlap_found and shift_id is not None:
            df1_updated.at[idx1, 'ShiftID'] = shift_id  # Assign ShiftID if an overlap exists
        else:
            df1_updated.at[idx1, 'ShiftID'] = None  # No match or overlap found

# Output or use df1_updated
df1_updated['Already Scheduled'] = np.where(df1_updated['ShiftID'].notnull(),'Y','N')
df1_updated = df1_updated[df1_updated['Already Scheduled'] != 'Y']
df1_updated

In [ ]:
#Remove Time if Break should be null
df1_updated['Lunch_Seconds'] = np.where(df1_updated['Lunch_Option'] == 'N',0,df1_updated['Lunch_Seconds'] )
df1_updated['BreakA_Seconds'] = np.where(df1_updated['BreakA_Option'] == 'N',0,df1_updated['BreakA_Seconds'] )
df1_updated['BreakB_Seconds'] = np.where(df1_updated['BreakB_Option'] == 'N',0,df1_updated['BreakB_Seconds'] )

df1_updated = df1_updated.reset_index(drop=True)

Client_Schedule_Blocker = """
select distinct * from (   
select Date_Blocked as 'Scheduled_Date', clientCode,  'Y' as 'Block_Date'
from reporting.wfm_block_date wb
where wb.clientCode is not null and wb.clientCode != ""
union all
select Date_Blocked as 'Scheduled_Date', clientCode,  'Y' as 'Block_Date'
from workforce.wfm_block_date wa
where wa.clientCode is not null and wa.clientCode != ""
) A """

Client_Schedule_Blocker_pd = pd.read_sql(Client_Schedule_Blocker, report_RO_DB) 
Client_Schedule_Blocker_pd = Client_Schedule_Blocker_pd.drop_duplicates().reset_index(drop=True)
sql="""SELECT ID as 'ProjectID', Code as 'clientCode'
FROM SS_focus.Project 
where active = 1
"""
Project_pd = pd.read_sql(sql, focus_db)

Schedule_Blocker_pd = pd.merge(Client_Schedule_Blocker_pd,Project_pd, on=['clientCode'],how='left' )

Schedule_Blocker_pd = Schedule_Blocker_pd[['Scheduled_Date','ProjectID','Block_Date']]

Schedule_Blocker_pd['ProjectID'] = Schedule_Blocker_pd['ProjectID'].fillna(0).astype(int)

# Test
# new_row = {
#     'Scheduled_Date': '2024-09-02',
#     'ProjectID': 42,
#     'Block_Date': 'Y'
# }

# # Append new row to the DataFrame
# Schedule_Blocker_pd = Schedule_Blocker_pd.append(new_row, ignore_index=True)
# Schedule_Blocker_pd

Schedule_Blocker_pd['Scheduled_Date'] = Schedule_Blocker_pd['Scheduled_Date'].fillna(0).astype(str)
df1_updated['Scheduled_Date'] = df1_updated['Scheduled_Date'].fillna(0).astype(str)
Schedule_Blocker_pd['ProjectID'] = Schedule_Blocker_pd['ProjectID'].fillna(0).astype(str)
df1_updated['ProjectID'] = df1_updated['ProjectID'].fillna(0).astype(str)
Schedule_Blocker_pd['Block_Date'] = Schedule_Blocker_pd['Block_Date'].fillna(0).astype(str)

Agents_Scheduled_Block = pd.merge(df1_updated,Schedule_Blocker_pd, on=['Scheduled_Date','ProjectID',],how='left')
Agents_Scheduled_Block

# Handles Block_date for Midnight shift

In [ ]:
df = Agents_Scheduled_Block

# Convert to datetime
df['Start_DateTime'] = pd.to_datetime(df['Start_DateTime'])
df['End_DateTime'] = pd.to_datetime(df['End_DateTime'])
df['Scheduled_Date'] = pd.to_datetime(df['Scheduled_Date'])

# Sort by MemberID, Scheduled_Date, Start_DateTime to ensure the order
df = df.sort_values(by=['MemberID', 'Scheduled_Date', 'Start_DateTime'])

# Initialize a list to collect all cleaned dataframes
cleaned_dfs = []

# Get the list of unique MemberIDs
member_ids = df['MemberID'].unique()

for member_id in member_ids:
    # Filter the dataframe for the current MemberID
    group = df[df['MemberID'] == member_id].copy()
    
    # Initialize a list to keep the indices of rows to be removed
    rows_to_remove = []
    
    # Iterate through the group to find rows with Block_Date = 'Y'
    for i in range(len(group)):
        current_row = group.iloc[i]
        
        # Check if the current row's Block_Date is 'Y'
        if current_row['Block_Date'] == 'Y':
            print(f"\nChecking row {i} for MemberID {member_id}: {current_row.to_dict()}")
            
            # Check the next row
            if i + 1 < len(group):
                next_row = group.iloc[i + 1]
                
                # If the next row ends at midnight, mark it for removal
                if next_row['End_DateTime'].strftime('%H:%M:%S') == '00:00:00':
                    print(f"Marking row {i + 1} for removal (ends at midnight): {next_row.to_dict()}")
                    rows_to_remove.append(next_row.name)
                
                # Check the row two days ahead for a midnight shift
                if i + 2 < len(group):
                    next_day_row = group.iloc[i + 2]
                    if next_day_row['Start_DateTime'].strftime('%H:%M:%S') == '00:00:00':
                        print(f"Marking row {i + 2} for removal (starts at midnight): {next_day_row.to_dict()}")
                        rows_to_remove.append(next_day_row.name)
    
    # Drop the rows identified for removal
    cleaned_group = group.drop(index=rows_to_remove)
    
    if not cleaned_group.empty:
        cleaned_dfs.append(cleaned_group)
    else:
        print(f"MemberID {member_id}: Cleaned group is empty, skipping append")

print(f"Number of cleaned dataframes: {len(cleaned_dfs)}")
if len(cleaned_dfs) == 0:
    print("No cleaned dataframes to concatenate")
else:
    df_cleaned = pd.concat(cleaned_dfs, ignore_index=True)
    Agents_Scheduled_Block = df_cleaned
    print("Final cleaned dataframe:")
    print(Agents_Scheduled_Block)

In [ ]:
Agents_Scheduled_Block = Agents_Scheduled_Block[Agents_Scheduled_Block['Block_Date'] != 'Y'].reset_index(drop=True)

Agents_Scheduled_Block = Agents_Scheduled_Block[Agents_Scheduled_Block['ProjectID'] != 0 ].reset_index(drop=True)
Agents_Scheduled_Block['Duration'] = Agents_Scheduled_Block['Duration'].dt.total_seconds()
Project_pd['ProjectID'] = Project_pd['ProjectID'].fillna(0).astype(str)
Agents_Scheduled_Block['ProjectID'] = Agents_Scheduled_Block['ProjectID'].fillna(0).astype(str)
Agents_Scheduled_Block



In [ ]:
# Agents_Scheduled_Block.to_excel("Agents_Scheduled_Block.xlsx")

### Run Schedules